<a href="https://colab.research.google.com/github/SJKIM2253/DeepLearning/blob/master/%EA%B3%BC%EC%A0%9C2(CIFAR10)_81_92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN (CIFAR10)
## 1. Library, Data Preprocessing 
## 2. Solve classification with Tensorflow

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Library

In [0]:
pip install tensorflow-gpu

In [62]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11770682144759883576
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3507060281994245647
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16702850229164939939
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11330115994
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5836083147067420304
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets.cifar10 import load_data

# Load data

In [0]:
(x_train, y_train), (x_test, y_test) = load_data()

# one-hot encoding
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10),axis=1)

# Hyper parameter setting

In [0]:
# hypter parameter setting
learning_rate = 1e-4
training_epochs = 1000
display_step = 15
batch_size = 30
activation = tf.nn.relu

# Model

In [0]:
def CNN_model(x):
    x_image = x
    
    W_conv1 = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, 32], stddev=5e-3))
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    batnor1 = tf.layers.batch_normalization(h_pool1)

    W_conv2 = tf.Variable(tf.truncated_normal(shape=[3, 3, 32, 64], stddev=1e-2))
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(batnor1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

    W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
    b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
    h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)
    batnor3 = tf.layers.batch_normalization(h_conv3)

    W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 256], stddev=5e-3))
    b_conv4 = tf.Variable(tf.constant(0.1, shape=[256])) 
    h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)
    batnor4 = tf.layers.batch_normalization(h_conv4)

    W_conv5 = tf.Variable(tf.truncated_normal(shape=[5, 5, 256, 512], stddev=1e-2))
    b_conv5 = tf.Variable(tf.constant(0.1, shape=[512]))
    h_conv5 = tf.nn.relu(tf.nn.conv2d(batnor4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)
    batnor5 = tf.layers.batch_normalization(h_conv5)

    W_fc1 = tf.Variable(tf.truncated_normal(shape=[8 * 8 * 512, 384], stddev=5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))

    h_conv5_flat = tf.reshape(batnor5, [-1, 8*8*512])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) 

    W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 10], stddev=5e-3))
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
    logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
    y_pred = tf.nn.softmax(logits)

    return y_pred, logits

tf_X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
tf_Y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

pred, logit = CNN_model(tf_X)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_Y, logits=logit))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(tf_Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Train

In [66]:
def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
  
    for i in range(10000):
        batch = next_batch(128, x_train, y_train_one_hot.eval())

        if i % 500 == 0:
            train_accuracy = accuracy.eval(feed_dict={tf_X: batch[0], tf_Y: batch[1], keep_prob: 1.0})
            loss_print = loss.eval(feed_dict={tf_X: batch[0], tf_Y: batch[1], keep_prob: 1.0})

            print("iteration: %d, accuracy: %f, loss: %f" % (i, train_accuracy, loss_print))
        sess.run(optimizer, feed_dict={tf_X: batch[0], tf_Y: batch[1], keep_prob: 0.25})

    test_accuracy = 0.0  
    for i in range(40):
        test_batch = next_batch(250, x_test, y_test_one_hot.eval())
        test_accuracy = test_accuracy + accuracy.eval(feed_dict={tf_X: test_batch[0], tf_Y: test_batch[1], keep_prob: 1.0})
    test_accuracy = test_accuracy / 40;
    print("Test accuracy: %f" % test_accuracy)

iteration: 0, accuracy: 0.078125, loss: 2.302204
iteration: 500, accuracy: 0.523438, loss: 1.438244
iteration: 1000, accuracy: 0.546875, loss: 1.165883
iteration: 1500, accuracy: 0.671875, loss: 1.011240
iteration: 2000, accuracy: 0.695312, loss: 0.789890
iteration: 2500, accuracy: 0.703125, loss: 0.877458
iteration: 3000, accuracy: 0.742188, loss: 0.847979
iteration: 3500, accuracy: 0.843750, loss: 0.572553
iteration: 4000, accuracy: 0.789062, loss: 0.596108
iteration: 4500, accuracy: 0.789062, loss: 0.610058
iteration: 5000, accuracy: 0.898438, loss: 0.301139
iteration: 5500, accuracy: 0.851562, loss: 0.417304
iteration: 6000, accuracy: 0.898438, loss: 0.271317
iteration: 6500, accuracy: 0.906250, loss: 0.261356
iteration: 7000, accuracy: 0.937500, loss: 0.245379
iteration: 7500, accuracy: 0.921875, loss: 0.212618
iteration: 8000, accuracy: 0.921875, loss: 0.178337
iteration: 8500, accuracy: 0.968750, loss: 0.100600
iteration: 9000, accuracy: 0.898438, loss: 0.225058
iteration: 9500,